# Enhanced CodeBERT for Swift Code Understanding

In this notebook, we fine-tune the [CodeBERT](https://github.com/microsoft/CodeBERT) model on the [Swift Code Intelligence dataset](https://huggingface.co/datasets/mvasiliniuc/iva-swift-codeint). CodeBERT is a pre-trained model specifically designed for programming languages, much like how BERT was pre-trained for natural language text. Created by Microsoft Research, CodeBERT can understand both programming language and natural language, making it ideal for code-related tasks.

Unlike the previous version that focused only on identifying Package.swift files, this enhanced version trains the model on the entire dataset by classifying Swift files into meaningful categories based on their purpose in a codebase.

## Overview

The process of fine-tuning CodeBERT involves:

1. **🔧 Setup**: Install necessary libraries and prepare our environment
2. **📥 Data Loading**: Load the Swift code dataset from Hugging Face
3. **🧹 Enhanced Preprocessing**: Prepare the data for training by categorizing files and tokenizing the code samples
4. **🧠 Model Training**: Fine-tune CodeBERT on our prepared data
5. **📊 Evaluation**: Assess how well our model performs
6. **📤 Export & Upload**: Save the model and upload it to Dropbox

Let's start by installing the necessary libraries:

In [ ]:
# Uninstall TensorFlow and install TensorFlow-cpu (better for Kaggle environment)
!pip uninstall -y tensorflow
!pip install tensorflow-cpu
# Install required libraries
!pip install transformers datasets evaluate torch scikit-learn tqdm dropbox requests


In [ ]:
# Important: These imports must be properly separated
import os
import json
import torch
import random
import numpy as np
import time
import gc
import re
import collections
from tqdm.auto import tqdm
from datasets import load_dataset, ClassLabel
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    RobertaForSequenceClassification,
    Trainer, 
    TrainingArguments,
    set_seed,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    get_scheduler
)

# Import AdamW from torch.optim instead of transformers.optimization
from torch.optim import AdamW
from transformers.trainer_utils import get_last_checkpoint

# Set a seed for reproducibility
set_seed(42)

# Add memory management function
def cleanup_memory():
    """Force garbage collection and clear CUDA cache if available."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Memory cleaned up.")


## Accelerator Detection and Configuration

Let's detect and configure the available accelerator (CPU, GPU, or TPU):

In [ ]:
# Check if GPU is available
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


## Dataset and Model Configuration

Let's define the model and dataset we'll be using:

In [ ]:
# Dataset configuration
DATASET_ID = "mvasiliniuc/iva-swift-codeint"

# Model configuration
MODEL_NAME = "microsoft/codebert-base"
MAX_LENGTH = 512
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 5
WARMUP_STEPS = 500
GRADIENT_ACCUMULATION_STEPS = 4

print("Using default configuration values.")


## Data Loading

Now let's load the Swift code dataset and examine its structure with proper error handling:

In [ ]:
# Function to load dataset with retry logic
def load_dataset_with_retry(dataset_id, max_retries=3, retry_delay=5):
    """Load a dataset with retry logic."""
    for attempt in range(max_retries):
        try:
            print(f"Loading dataset (attempt {attempt+1}/{max_retries})...")
            data = load_dataset(dataset_id, trust_remote_code=True)
            print(f"Dataset loaded successfully with {len(data['train'])} examples")
            return data
        except Exception as e:
            print(f"Error loading dataset (attempt {attempt+1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print("Maximum retries reached. Could not load dataset.")
                raise

# Make sure dataset ID is defined (in case previous cell didn't execute)
if 'DATASET_ID' not in globals():
    print("Warning: DATASET_ID not found. Using default value.")
    DATASET_ID = "mvasiliniuc/iva-swift-codeint"  # Default value as fallback
    MAX_LENGTH = 384
    MODEL_ID = "microsoft/codebert-base"
    TRAIN_BATCH_SIZE = 8
    EVAL_BATCH_SIZE = 16
    GRADIENT_ACCUMULATION_STEPS = 4
    print("Using default configuration values.")

# Load the dataset with retry logic
try:
    print(f"Loading dataset: {DATASET_ID}")
    data = load_dataset_with_retry(DATASET_ID)
    print("Dataset structure:")
    print(data)
except Exception as e:
    print(f"Fatal error loading dataset: {e}")
    raise


In [ ]:
# Verify dataset structure and column names
def verify_dataset_structure(dataset):
    """Verify that the dataset has the expected structure and columns."""
    required_columns = ['repo_name', 'path', 'content']
    if 'train' not in dataset:
        print("WARNING: Dataset does not have a 'train' split.")
        return False
    
    missing_columns = [col for col in required_columns if col not in dataset['train'].column_names]
    if missing_columns:
        print(f"WARNING: Dataset is missing required columns: {missing_columns}")
        return False
    
    print("Dataset structure verification passed.")
    return True

# Verify dataset structure
dataset_valid = verify_dataset_structure(data)
if not dataset_valid:
    print("Dataset structure is not as expected. Proceeding with caution.")


In [ ]:
# Let's take a look at an example from the dataset
try:
    if 'train' in data:
        example = data['train'][0]
    else:
        example = data[list(data.keys())[0]][0]
    
    print("Example features:")
    for key, value in example.items():
        if isinstance(value, str) and len(value) > 100:
            print(f"{key}: {value[:100]}...")
        else:
            print(f"{key}: {value}")
except Exception as e:
    print(f"Error exploring dataset example: {e}")


## Loading the CodeBERT Tokenizer

Now, let's load the CodeBERT tokenizer, which has been specially trained to handle code tokens:

In [ ]:
# Load the CodeBERT tokenizer with error handling
try:
    # Use MODEL_NAME instead of MODEL_ID to match the variable defined earlier
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    print(f"Tokenizer vocabulary size: {len(tokenizer)}")
    print(f"Tokenizer type: {tokenizer.__class__.__name__}")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise


## Enhanced Data Preparation

Instead of focusing only on Package.swift files, we'll create a more meaningful multi-class classification task that categorizes Swift files based on their purpose in a codebase. This approach utilizes the entire dataset and provides more valuable insights into code understanding.

We'll categorize files into the following classes:
1. **Models** - Data structures and model definitions
2. **Views** - UI related files
3. **Controllers** - Application logic
4. **Utilities** - Helper functions and extensions
5. **Tests** - Test files
6. **Configuration** - Package and configuration files

In [ ]:
def extract_file_type(path):
    """
    Extract the file type/category based on the file path and naming conventions in Swift projects.
    
    Args:
        path (str): The file path
        
    Returns:
        int: The category label (0-5)
    """
    path_lower = path.lower()
    filename = path.split('/')[-1].lower()
    
    # Category 0: Models - Data structures and model definitions
    if ('model' in path_lower or 
        'struct' in path_lower or 
        'entity' in path_lower or
        'data' in path_lower and 'class' in path_lower):
        return 0
    
    # Category 1: Views - UI related files
    elif ('view' in path_lower or 
          'ui' in path_lower or 
          'screen' in path_lower or 
          'page' in path_lower or
          'controller' in path_lower and 'view' in path_lower):
        return 1
    
    # Category 2: Controllers - Application logic
    elif ('controller' in path_lower or 
          'manager' in path_lower or 
          'coordinator' in path_lower or
          'service' in path_lower):
        return 2
    
    # Category 3: Utilities - Helper functions and extensions
    elif ('util' in path_lower or 
          'helper' in path_lower or 
          'extension' in path_lower or
          'common' in path_lower):
        return 3
    
    # Category 4: Tests - Test files
    elif ('test' in path_lower or 
          'spec' in path_lower or 
          'mock' in path_lower):
        return 4
    
    # Category 5: Configuration - Package and configuration files
    elif ('package.swift' in path_lower or 
          'config' in path_lower or 
          'settings' in path_lower or
          'info.plist' in path_lower):
        return 5
    
    # Default to category 3 (Utilities) if no clear category is found
    return 3

def analyze_content_for_category(content):
    """
    Analyze file content to help determine its category when path-based classification is ambiguous.
    
    Args:
        content (str): The file content
        
    Returns:
        int: The suggested category based on content analysis
    """
    content_lower = content.lower()
    
    # Check for model patterns
    if (re.search(r'struct\s+\w+', content) or 
        re.search(r'class\s+\w+\s*:\s*\w*codable', content_lower) or
        'encodable' in content_lower or 'decodable' in content_lower):
        return 0
    
    # Check for view patterns
    elif ('uiview' in content_lower or 
          'uitableview' in content_lower or 
          'uicollectionview' in content_lower or
          'swiftui' in content_lower or
          'view {' in content_lower):
        return 1
    
    # Check for controller patterns
    elif ('viewcontroller' in content_lower or 
          'uiviewcontroller' in content_lower or
          'navigationcontroller' in content_lower or
          'viewdidload' in content_lower):
        return 2
    
    # Check for utility patterns
    elif ('extension' in content_lower or 
          'func ' in content and not 'class' in content_lower[:100] or
          'protocol' in content_lower):
        return 3
    
    # Check for test patterns
    elif ('xctest' in content_lower or 
          'testcase' in content_lower or
          'func test' in content_lower):
        return 4
    
    # Check for configuration patterns
    elif ('package(' in content_lower or 
          'dependencies' in content_lower and 'package' in content_lower or
          'products' in content_lower and 'targets' in content_lower):
        return 5
    
    # Default to -1 (undetermined)
    return -1

def enhanced_add_labels(example):
    """
    Enhanced labeling function that categorizes Swift files based on their purpose.
    
    Categories:
    0: Models - Data structures and model definitions
    1: Views - UI related files
    2: Controllers - Application logic
    3: Utilities - Helper functions and extensions
    4: Tests - Test files
    5: Configuration - Package and configuration files
    
    Args:
        example: Dataset example with 'path' and 'content' fields
        
    Returns:
        example: The example with added 'label' field
    """
    # First try to determine category from path
    path_category = extract_file_type(example['path'])
    
    # If the path-based category is ambiguous (category 3 - Utilities is our default),
    # try to analyze the content for a more specific category
    if path_category == 3:
        content_category = analyze_content_for_category(example['content'])
        # Only use content category if it's determined (-1 means undetermined)
        if content_category != -1:
            example['label'] = content_category
        else:
            example['label'] = path_category
    else:
        example['label'] = path_category
    
    return example


In [ ]:
try:
    # Apply the enhanced labeling function
    labeled_data = data['train'].map(enhanced_add_labels)
    
    # Check the distribution of labels
    all_labels = labeled_data['label']
    label_counter = collections.Counter(all_labels)
    
    print("Label distribution:")
    for label, count in label_counter.items():
        category_names = {
            0: "Models",
            1: "Views",
            2: "Controllers",
            3: "Utilities",
            4: "Tests",
            5: "Configuration"
        }
        category_name = category_names.get(label, f"Category {label}")
        print(f"Label {label} ({category_name}): {count} examples ({count/len(labeled_data)*100:.2f}%)")
    
    # Check for label imbalance
    min_label_count = min(label_counter.values())
    max_label_count = max(label_counter.values())
    imbalance_ratio = max_label_count / min_label_count if min_label_count > 0 else float('inf')
    
    if imbalance_ratio > 10:
        print(f"WARNING: Severe label imbalance detected (ratio: {imbalance_ratio:.2f}). Consider using class weights or resampling.")
    elif imbalance_ratio > 3:
        print(f"WARNING: Moderate label imbalance detected (ratio: {imbalance_ratio:.2f}). Consider using class weights.")
        
except Exception as e:
    print(f"Error preparing dataset: {e}")
    raise


Now let's split our data into training and validation sets with stratification to maintain label distribution:

In [ ]:
try:
    # Get unique labels
    unique_labels = sorted(set(labeled_data["label"]))
    num_labels = len(unique_labels)
    
    # Create a new dataset with ClassLabel feature
    labeled_data = labeled_data.cast_column("label", ClassLabel(num_classes=num_labels, names=[str(i) for i in unique_labels]))
    
    # First split: Create train and temp sets (temp will be split into val and test)  
    train_temp_split = labeled_data.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')
    train_data = train_temp_split['train']
    
    # Second split: Split temp into validation and test sets
    val_test_split = train_temp_split['test'].train_test_split(test_size=0.5, seed=42, stratify_by_column='label')
    val_data = val_test_split['train']
    test_data = val_test_split['test']
    
    # Verify label distribution after split
    train_label_counter = collections.Counter(train_data['label'])
    val_label_counter = collections.Counter(val_data['label'])
    test_label_counter = collections.Counter(test_data['label'])
    
    print(f"Training set size: {len(train_data)}")
    print(f"Training label distribution: {dict(train_label_counter)}")
    
    print(f"Validation set size: {len(val_data)}")
    print(f"Validation label distribution: {dict(val_label_counter)}")
    
    print(f"Test set size: {len(test_data)}")
    print(f"Test label distribution: {dict(test_label_counter)}")
    
except Exception as e:
    print(f"Error splitting dataset: {e}")
    raise


## Tokenization

Now let's tokenize our data for the model:

In [ ]:
def tokenize_function(examples):
    # Tokenize the texts with proper padding and truncation
    return tokenizer(
        examples['content'],  # Use 'content' column instead of 'func'
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )

# Apply tokenization to the datasets
tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_val_data = val_data.map(tokenize_function, batched=True)
tokenized_test_data = test_data.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print(f"Tokenized {len(tokenized_train_data)} training examples")
print(f"Tokenized {len(tokenized_val_data)} validation examples")
print(f"Tokenized {len(tokenized_test_data)} test examples")


## Model Setup

Now let's set up the CodeBERT model for our multi-class classification task:

In [ ]:
try:
    # Load the model with the correct number of labels
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, 
        num_labels=num_labels,
        problem_type="single_label_classification"
    )
    
    # Move model to the appropriate device
    model.to(device)
    
    print(f"Model loaded with {num_labels} output classes")
    print(f"Model type: {model.__class__.__name__}")
    
except Exception as e:
    print(f"Error loading model: {e}")
    raise


## Training Setup

Let's set up the training configuration:

In [ ]:
# Define metrics computation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    
    # Calculate precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    # Return all metrics
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU is available
    dataloader_num_workers=4,  # Use multiple workers for faster data loading
    dataloader_pin_memory=True,  # Pin memory for faster data transfer to GPU
    report_to="none"  # Disable reporting to avoid potential issues
)

# Create data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set up early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

print("Training setup complete")


## Model Training

Now let's train the model:

In [ ]:
try:
    print("Starting training...")
    train_result = trainer.train()
    
    # Print training results
    print(f"Training completed in {train_result.metrics['train_runtime']:.2f} seconds")
    print(f"Training loss: {train_result.metrics['train_loss']:.4f}")
    
    # Save the model
    trainer.save_model("./final_model")
    print("Model saved to ./final_model")
    
    # Clean up memory
    cleanup_memory()
    
except Exception as e:
    print(f"Error during training: {e}")
    raise


## Model Evaluation

Let's evaluate our model on the test set:

In [ ]:
try:
    print("Evaluating model on test set...")
    test_results = trainer.evaluate(tokenized_test_data)
    
    # Print evaluation results
    print("Test results:")
    for key, value in test_results.items():
        print(f"{key}: {value:.4f}")
    
    # Create a confusion matrix
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    import matplotlib.pyplot as plt
    
    # Get predictions
    predictions = trainer.predict(tokenized_test_data)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids
    
    # Create confusion matrix
    cm = confusion_matrix(labels, preds)
    
    # Define class names
    class_names = ["Models", "Views", "Controllers", "Utilities", "Tests", "Configuration"]
    
    # Display confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(ax=ax, cmap=plt.cm.Blues, values_format='d')
    plt.title('Confusion Matrix')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error during evaluation: {e}")
    raise


## Prediction Examples

Let's look at some examples of model predictions:

In [ ]:
try:
    # Get a few examples from the test set
    num_examples = 5
    examples = test_data.select(range(num_examples))
    
    # Tokenize examples
    inputs = tokenizer(examples['content'], padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_classes = torch.argmax(predictions, dim=-1).cpu().numpy()
    
    # Define class names
    class_names = ["Models", "Views", "Controllers", "Utilities", "Tests", "Configuration"]
    
    # Print predictions
    print("Example predictions:")
    for i in range(num_examples):
        print(f"\nExample {i+1}:")
        print(f"File path: {examples['path'][i]}")
        print(f"Content preview: {examples['content'][i][:100]}...")
        print(f"Predicted class: {predicted_classes[i]} ({class_names[predicted_classes[i]]})")
        print(f"True class: {examples['label'][i]} ({class_names[examples['label'][i]]})")
        print(f"Confidence: {predictions[i][predicted_classes[i]].item():.4f}")
        
        # Print top 3 predictions
        top_3 = torch.topk(predictions[i], 3)
        print("Top 3 predictions:")
        for j in range(3):
            idx = top_3.indices[j].item()
            prob = top_3.values[j].item()
            print(f"  {class_names[idx]}: {prob:.4f}")
    
except Exception as e:
    print(f"Error during prediction examples: {e}")
    raise


## Model Export

Let's save and export our model:

In [ ]:
try:
    # Save model and tokenizer
    output_dir = "./swift_codebert_classifier"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Save class names and other metadata
    metadata = {
        "class_names": class_names,
        "num_classes": num_labels,
        "max_length": MAX_LENGTH,
        "model_name": MODEL_NAME,
        "dataset": DATASET_ID,
        "accuracy": float(test_results["eval_accuracy"]),
        "f1": float(test_results["eval_f1"]),
        "precision": float(test_results["eval_precision"]),
        "recall": float(test_results["eval_recall"])
    }
    
    with open(f"{output_dir}/metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)
    
    print(f"Model, tokenizer, and metadata saved to {output_dir}")
    
    # Create a zip file for easy download
    import shutil
    shutil.make_archive("swift_codebert_classifier", "zip", ".", output_dir)
    print("Model package created: swift_codebert_classifier.zip")
    
except Exception as e:
    print(f"Error saving model: {e}")
    raise


## Conclusion

We've successfully enhanced the CodeBERT training process to utilize the entire Swift code dataset instead of focusing only on Package.swift files. Our model now classifies Swift code files into meaningful categories based on their purpose in a codebase:

1. **Models** - Data structures and model definitions
2. **Views** - UI related files
3. **Controllers** - Application logic
4. **Utilities** - Helper functions and extensions
5. **Tests** - Test files
6. **Configuration** - Package and configuration files

This multi-class classification approach provides more valuable insights for code understanding tasks and makes better use of the available data. The model can be used for various code intelligence tasks such as:

- Automatically categorizing new code files
- Suggesting file organization in large codebases
- Identifying misplaced code (e.g., model logic in controller files)
- Assisting in code navigation and understanding

The same approach can be extended to other programming languages and tasks, such as code search, code completion, and bug detection.